In [1]:
import numpy as np
import pandas as pd
from transformers import T5Tokenizer, TFT5ForConditionalGeneration
import tensorflow as tf
import gc
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

/home/davidsolow/.pyenv/versions/3.11.1/envs/med-abbrev-mystery/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2024-07-12 22:47:27.344541: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:479] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-07-12 22:47:27.358888: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:10575] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-07-12 22:47:27.358917: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1442] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-07-12 22:47:27.369250: I tensorflow/core/platform/cpu_feature

Num GPUs Available:  1


2024-07-12 22:47:28.748607: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:998] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-07-12 22:47:28.752378: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:998] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-07-12 22:47:28.755328: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:998] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-

In [2]:
max_len=200

def process_location(location):
    return list(map(int, eval(location)))[0]

def process_label(label):
    return eval(label)[0]

# Reading and processing training data
train_df = pd.read_csv('data/medal_train.csv')
train_df['location'] = train_df['location'].map(process_location)
train_df['label'] = train_df['label'].map(process_label)
train_df['abbreviation'] = train_df.apply(lambda row: row['text'].split()[row['location']], axis=1)
train_df = train_df[train_df['location'] <= max_len-51]

# Reading and processing validation data
val_df = pd.read_csv('data/medal_val.csv')
val_df['location'] = val_df['location'].map(process_location)
val_df['label'] = val_df['label'].map(process_label)
val_df['abbreviation'] = val_df.apply(lambda row: row['text'].split()[row['location']], axis=1)
val_df = val_df[val_df['location'] <= max_len-51]

# Reading and processing test data
test_df = pd.read_csv('data/medal_test.csv')
test_df['location'] = test_df['location'].map(process_location)
test_df['label'] = test_df['label'].map(process_label)
test_df['abbreviation'] = test_df.apply(lambda row: row['text'].split()[row['location']], axis=1)
test_df = test_df[test_df['location'] <= max_len-51]

train_df.head()

,Unnamed: 0,abstract_id,text,location,label,abbreviation
0,0,14145090,velvet antlers vas are commonly used in tradit...,63,transverse aortic constriction,TAC
1,1,1900667,the clinical features of our cases demonstrate...,85,hodgkins lymphoma,HD
2,2,8625554,ceftobiprole bpr is an investigational cephalo...,90,methicillinsusceptible s aureus,MSSA
3,3,8157202,we have taken a basic biologic RPA to elucidat...,26,parathyroid hormonerelated protein,PTHrP
5,5,4846741,gb virus c gbvc or HGV hgv is transmitted by t...,5,hepatitis g virus,HGV


In [3]:
train_text = train_df['text'].values[:10000]
train_abbrv = train_df['abbreviation'].values[:10000]
train_loc = train_df['location'].values[:10000]
train_label = train_df['label'].values[:10000]

val_text = val_df['text'].values[:1000]
val_abbrv = val_df['abbreviation'].values[:100]
val_loc = val_df['location'].values[:1000]
val_label = val_df['label'].values[:1000]

test_text = test_df['text'].values[:100]
test_abbrv = test_df['abbreviation'].values[:1000]
test_loc = test_df['location'].values[:1000]
test_label = test_df['label'].values[:1000]

del train_df
del val_df
del test_df
gc.collect()

train_text[:2]

array(['velvet antlers vas are commonly used in traditional chinese medicine and invigorant and contain many PET components for health promotion the velvet antler peptide svap is one of active components in vas based on structural study the svap interacts with tgfÎ² receptors and disrupts the tgfÎ² pathway we hypothesized that svap prevents cardiac fibrosis from pressure overload by blocking tgfÎ² signaling SDRs underwent TAC tac or a sham operation T3 one month rats received either svap mgkgday or vehicle for an additional one month tac surgery induced significant cardiac dysfunction FB activation and fibrosis these effects were improved by treatment with svap in the heart tissue tac remarkably increased the expression of tgfÎ² and connective tissue growth factor ctgf ROS species C2 and the phosphorylation C2 of smad and ERK kinases erk svap inhibited the increases in reactive oxygen species C2 ctgf expression and the phosphorylation of smad and erk but not tgfÎ² expression in culture

In [4]:
# acronyms = train_abbrv.tolist() + val_abbrv.tolist() + test_abbrv.tolist()
# expansions = train_label.tolist() + val_label.tolist() + test_label.tolist()
# expansions = [word for expansion in expansions for word in expansion.split()]
# new_terms = list(set(acronyms + expansions))

model_checkpoint = "t5-small"
tokenizer = T5Tokenizer.from_pretrained(model_checkpoint)
# num_new_tokens = tokenizer.add_tokens(new_terms)
# print(f"Number of new tokens added: {num_new_tokens}")


You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [5]:
def tokenize(texts, locations, expansions, tokenizer=tokenizer, max_len=max_len):
    input_text = []
    label_text = []
    task_prefix = "expand acronym: "

    for text, loc, expansion in zip(texts, locations, expansions):
        text_with_task = task_prefix + text
        input_text.append(text_with_task)
        label = text.split()
        label = label[:loc] + expansion.split() + label[loc+1:]
        label = ' '.join(label)
        label_text.append(label)
        
    encoded_input = tokenizer(
        input_text,
        add_special_tokens=True,
        max_length=max_len,
        padding='max_length',
        truncation=True,
        return_attention_mask=True,
        return_tensors='tf'
    )
    
    encoded_output = tokenizer(
        label_text,
        add_special_tokens=True,
        max_length=max_len,
        padding='max_length',
        truncation=True,
        return_attention_mask=False,
        return_tensors='tf'
    )

    print("First text:\n", texts[0])
    print("First location:", locations[0])
    print("First acronym:", texts[0].split()[locations[0]])
    print("First expansion:", expansions[0])
    print("First text with expansion:\n", label_text[0])
    decoded_tokens = tokenizer.convert_ids_to_tokens(encoded_output['input_ids'][0].numpy())
    print("First label with expansion:\n", tokenizer.convert_tokens_to_string(decoded_tokens))

    return encoded_input['input_ids'], encoded_input['attention_mask'], encoded_output['input_ids']

In [6]:
print("Train----------------------------")
train_input_ids, train_attention_masks, train_decoder_input_ids = tokenize(
    train_text, train_loc, train_label, tokenizer, max_len)

print("Val------------------------------")
val_input_ids, val_attention_masks, val_decoder_input_ids = tokenize(
    val_text, val_loc, val_label, tokenizer, max_len)

print("Test-----------------------------")
test_input_ids, test_attention_masks, test_decoder_input_ids = tokenize(
    test_text, test_loc, test_label, tokenizer, max_len)


del train_text
del train_loc
del train_label
del val_text
del val_loc
del val_label
del test_text
del test_loc
del test_label

gc.collect()

Train----------------------------


2024-07-12 22:49:58.187204: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:998] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-07-12 22:49:58.197844: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:998] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-07-12 22:49:58.205529: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:998] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-

First text:
 velvet antlers vas are commonly used in traditional chinese medicine and invigorant and contain many PET components for health promotion the velvet antler peptide svap is one of active components in vas based on structural study the svap interacts with tgfÎ² receptors and disrupts the tgfÎ² pathway we hypothesized that svap prevents cardiac fibrosis from pressure overload by blocking tgfÎ² signaling SDRs underwent TAC tac or a sham operation T3 one month rats received either svap mgkgday or vehicle for an additional one month tac surgery induced significant cardiac dysfunction FB activation and fibrosis these effects were improved by treatment with svap in the heart tissue tac remarkably increased the expression of tgfÎ² and connective tissue growth factor ctgf ROS species C2 and the phosphorylation C2 of smad and ERK kinases erk svap inhibited the increases in reactive oxygen species C2 ctgf expression and the phosphorylation of smad and erk but not tgfÎ² expression in cu

0

In [11]:
model = TFT5ForConditionalGeneration.from_pretrained(model_checkpoint)
model.trainable = True
optimizer = tf.keras.optimizers.Adam(learning_rate=0.00005)
model.compile(optimizer=optimizer)
model.summary()

All PyTorch model weights were used when initializing TFT5ForConditionalGeneration.

All the weights of TFT5ForConditionalGeneration were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFT5ForConditionalGeneration for predictions without further training.


Model: "tft5_for_conditional_generation_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 shared (Embedding)          multiple                  16449536  
                                                                 
 encoder (TFT5MainLayer)     multiple                  35330816  
                                                                 
 decoder (TFT5MainLayer)     multiple                  41625344  
                                                                 
Total params: 60506624 (230.81 MB)
Trainable params: 60506624 (230.81 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [12]:
history = model.fit(
    {'input_ids': train_input_ids.numpy(),
     'attention_mask': train_attention_masks.numpy(),
     'decoder_input_ids': train_decoder_input_ids.numpy()},
    train_decoder_input_ids.numpy(),
    validation_data=({'input_ids': val_input_ids.numpy(),
                      'attention_mask': val_attention_masks.numpy(),
                      'decoder_input_ids': val_decoder_input_ids.numpy()},
                      val_decoder_input_ids.numpy()),
    epochs=5,
    batch_size=16
)


Epoch 1/5


I0000 00:00:1720839353.238212  352123 service.cc:145] XLA service 0x7d9de2d69460 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1720839353.238234  352123 service.cc:153]   StreamExecutor device (0): NVIDIA GeForce RTX 3080, Compute Capability 8.6
2024-07-12 22:55:53.242571: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:268] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2024-07-12 22:55:53.258327: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:465] Loaded cuDNN version 8902
I0000 00:00:1720839353.294737  352123 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


625/625 [==============================] - 120s 137ms/step - loss: 1.3011 - val_loss: 0.0313
Epoch 2/5
625/625 [==============================] - 76s 122ms/step - loss: 0.0762 - val_loss: 0.0062
Epoch 3/5
625/625 [==============================] - 76s 122ms/step - loss: 0.0306 - val_loss: 0.0026
Epoch 4/5
625/625 [==============================] - 78s 124ms/step - loss: 0.0159 - val_loss: 0.0011
Epoch 5/5
625/625 [==============================] - 76s 122ms/step - loss: 0.0096 - val_loss: 6.9843e-04


In [22]:
test_loss = model.evaluate(
    {'input_ids': test_input_ids, 'attention_mask': test_attention_masks, 'decoder_input_ids': test_decoder_input_ids},
    test_decoder_input_ids
)

print("Test Loss:", test_loss)

y_pred = model.generate(test_input_ids,
                        num_beams=3,
                        no_repeat_ngram_size=3,
                        min_length=10,
                        max_length=40)
y_true = test_decoder_input_ids


4/4 [==============================] - 0s 65ms/step - loss: 5.4284e-04
Test Loss: 0.0005428431322798133
First X Test:
 expand acronym: we developed an animal model of chronic allergic airway disease by repeatedly exposing nine sheep to tracheal instillation of ascaris antigen until stable increase in RL at three times control in six reactive sheep group c was obtained they were then compared to the three nonreactive sheep group b and a control group of eight sheep exposed to saline only group a in terms of pulmonary CF tests and bronchoalveolar lavage bal analyses RL was cm holsec in group a in group b and in group c trapping volume FRC by plethysmography and by helium rebreathing technique was l in group a in group b and in group c UP resistance at PF did not differ between any two CG but UP resistance near residual volume was cm holsec in a in b and in</s>
First Y True:
 we developed an animal model of chronic allergic airway disease by repeatedly exposing nine sheep to tracheal inst

In [23]:
print("First X Test:\n", tokenizer.convert_tokens_to_string(tokenizer.convert_ids_to_tokens(test_input_ids[0].numpy())))
print("First Y True:\n", tokenizer.convert_tokens_to_string(tokenizer.convert_ids_to_tokens(y_true[0].numpy())))
print([tokenizer.decode(t, skip_special_tokens=True, 
                           clean_up_tokenization_spaces=False) for t in y_pred[0]])

First X Test:
 expand acronym: we developed an animal model of chronic allergic airway disease by repeatedly exposing nine sheep to tracheal instillation of ascaris antigen until stable increase in RL at three times control in six reactive sheep group c was obtained they were then compared to the three nonreactive sheep group b and a control group of eight sheep exposed to saline only group a in terms of pulmonary CF tests and bronchoalveolar lavage bal analyses RL was cm holsec in group a in group b and in group c trapping volume FRC by plethysmography and by helium rebreathing technique was l in group a in group b and in group c UP resistance at PF did not differ between any two CG but UP resistance near residual volume was cm holsec in a in b and in</s>
First Y True:
 we developed an animal model of chronic allergic airway disease by repeatedly exposing nine sheep to tracheal instillation of ascaris antigen until stable increase in RL at three times control in six reactive sheep gro